In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
# Setting up connection
con = sqlite3.connect("../../basketball.sqlite")
cur = con.cursor()

In [3]:
# Parameters
rolling_games = 20
season_start = 2011
season_end = 2021
cleaned = True
if cleaned:
    table_name = 'GoodGames'
else:
    table_name = 'Game'

In [4]:
con.execute('''DROP TABLE IF EXISTS GoodGames''')

fullQuery = F'''SELECT DISTINCT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home, GAME_DATE,
            GAME_ID, TEAM_ID_HOME, TEAM_ABBREVIATION_HOME, TEAM_CITY_HOME, TEAM_ID_AWAY, TEAM_ABBREVIATION_AWAY, Season,
            FGM_HOME, FGA_HOME, FG3M_HOME, FTM_HOME, FTA_HOME, OREB_HOME, DREB_HOME, AST_HOME, STL_HOME,
            TOV_HOME, PF_HOME, PTS_HOME, WL_HOME, FGM_AWAY, FGA_AWAY, FG3M_AWAY, FTM_AWAY, OREB_AWAY, DREB_AWAY, 
            AST_AWAY, STL_AWAY, TOV_AWAY, PF_AWAY, PTS_AWAY, FTA_AWAY, WL_AWAY
    FROM Game
    WHERE Season >= {season_start} AND Season <= {season_end}
    AND TEAM_CITY_HOME NOT Null
    AND WL_HOME NOT Null'''

tempDF = pd.read_sql_query(fullQuery,con)
originalCount = len(tempDF)
floatColumns = ['FGM_HOME', 'FGA_HOME', 'FG3M_HOME', 'FTM_HOME', 'FTA_HOME', 'OREB_HOME', 'DREB_HOME', 'AST_HOME', 'STL_HOME',
                'TOV_HOME', 'PF_HOME', 'PTS_HOME', 'FGA_AWAY', 'FGM_AWAY', 'FG3M_AWAY', 'FTM_AWAY', 'OREB_AWAY', 'DREB_AWAY',
               'AST_AWAY', 'STL_AWAY', 'TOV_AWAY', 'PF_AWAY', 'PTS_AWAY', 'FTA_AWAY']
for col in floatColumns:
    tempDF = tempDF[(np.abs(tempDF[col].astype(float) - tempDF[col].astype(float).mean()) <= (3*tempDF[col].astype(float).std()))]
print("Original Row Count:", originalCount)
print("New Row Count:", len(tempDF))
tempDF.to_sql('GoodGames', con);

Original Row Count: 11698
New Row Count: 10804


In [5]:
# Get Team list
home_team_query = f'''SELECT DISTINCT TEAM_ID_HOME
    FROM GoodGames
    WHERE Season >= {season_start} AND Season <= {season_end}'''

cur.execute(home_team_query)
home_team_list = cur.fetchall()
home_team_list = [int(i[0]) for i in home_team_list]


away_team_query = f'''SELECT DISTINCT TEAM_ID_AWAY
    FROM GoodGames
    WHERE Season >= {season_start} AND Season <= {season_end}'''

cur.execute(away_team_query)
away_team_list = cur.fetchall()
away_team_list = [int(i[0]) for i in away_team_list]

In [6]:
def home_rolling_query(team, season_start, season_end, prev_games, table_name):
    return f'''SELECT DISTINCT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
            GAME_ID AS GAME_ID, TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
            AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
            AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
            AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
            AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
            AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
            AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
            AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
            AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
            AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
            AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
            AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
            AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
            CASE
                WHEN WL_HOME = 'W'
                THEN 1
                ELSE 0
            END AS home_win
    FROM {table_name}
    WHERE Season >= {season_start} AND Season <= {season_end} AND TEAM_ID_HOME = {team}
    AND TEAM_CITY_HOME NOT Null
    AND WL_HOME NOT Null
    '''

In [7]:
home_df = pd.DataFrame()
for team in home_team_list:
    team_df = pd.read_sql_query(home_rolling_query(team, season_start, season_end, rolling_games, table_name),con)
    if home_df is None:
        home_df = team_df
    else:
        home_df = pd.concat([home_df, team_df])

In [8]:
home_df = home_df.dropna(axis=0)

In [9]:
def away_rolling_query(team, season_start, season_end, prev_games, table_name):
    return f'''SELECT DISTINCT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        GAME_ID AS GAME_ID, TEAM_ID_AWAY AS TEAM_AWAY_ID, TEAM_ABBREVIATION_AWAY AS TEAM_ABBREVIATION_AWAY,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN {prev_games} PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM {table_name}
    WHERE Season >= {season_start} AND Season <= {season_end} AND TEAM_ID_AWAY = {team}
    AND TEAM_CITY_HOME NOT Null
    AND WL_HOME NOT Null'''

In [10]:
away_df = pd.DataFrame()
for team in away_team_list:
    team_df = pd.read_sql_query(away_rolling_query(team, season_start, season_end, rolling_games, table_name),con)
    if away_df is None:
        away_df = team_df
    else:
        away_df = pd.concat([away_df, team_df])

In [11]:
away_df = away_df.dropna(axis=0)

In [12]:
games_df = pd.merge(home_df,away_df,how='inner', on='GAME_ID')

In [13]:
games_df.head()

,Game_Num_Home,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,FGM_HOME_avg,FGA_HOME_avg,FG3M_HOME_avg,FTM_HOME_avg,FTA_HOME_avg,OREB_HOME_avg,...,FTM_AWAY_avg,FTA_AWAY_avg,OREB_AWAY_avg,DREB_AWAY_avg,AST_AWAY_avg,STL_AWAY_avg,TOV_AWAY_avg,PF_AWAY_avg,PTS_AWAY_avg,away_win
0,2,0021100017,1610612744,GSW,32.000000,82.000000,5.000000,17.000000,24.000000,17.000000,...,9.000000,14.000000,16.000000,25.000000,21.000000,14.000000,14.000000,21.000000,88.000000,0
1,4,0021100056,1610612744,GSW,34.333333,78.666667,5.000000,18.666667,26.333333,11.666667,...,16.666667,22.000000,9.333333,35.666667,20.666667,8.000000,15.000000,22.000000,101.666667,1
2,5,0021100119,1610612744,GSW,34.250000,81.250000,4.750000,15.750000,22.750000,12.750000,...,16.500000,21.000000,16.000000,29.000000,15.500000,9.500000,8.500000,26.500000,80.000000,1
3,6,0021100142,1610612744,GSW,33.200000,79.000000,5.200000,17.000000,23.000000,12.200000,...,19.000000,25.500000,9.750000,32.250000,23.250000,10.500000,17.750000,23.250000,101.250000,0
4,7,0021100226,1610612744,GSW,34.000000,81.666667,5.833333,18.500000,24.166667,13.166667,...,22.000000,27.571429,11.142857,29.142857,17.571429,9.428571,16.142857,21.428571,91.000000,1


In [14]:
col_order = ['home_win','FGM_HOME_avg', 'FGA_HOME_avg', 'FG3M_HOME_avg', 'FTM_HOME_avg', 'FTA_HOME_avg', 'OREB_HOME_avg', 
             'DREB_HOME_avg', 'AST_HOME_avg', 'STL_HOME_avg', 'TOV_HOME_avg', 'PF_HOME_avg', 'PTS_HOME_avg', 'FGM_AWAY_avg', 
             'FGA_AWAY_avg', 'FG3M_AWAY_avg', 'FTM_AWAY_avg', 'FTA_AWAY_avg', 'OREB_AWAY_avg', 'DREB_AWAY_avg', 
             'AST_AWAY_avg', 'STL_AWAY_avg', 'TOV_AWAY_avg', 'PF_AWAY_avg', 'PTS_AWAY_avg']

In [15]:
games_df = games_df[col_order]

In [16]:
if cleaned:
    outfile_name = f"../../data/Rolling{rolling_games}GamesCleaned.csv"
else:
    outfile_name = f"../../data/Rolling{rolling_games}Games.csv"
games_df.to_csv(outfile_name, index=False)